In [64]:
import numpy as np
import pandas as pd
from PIL import Image

import cv2 

%pylab inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import os


from sklearn import preprocessing
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras import models
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input

from tensorflow import keras

Populating the interactive namespace from numpy and matplotlib


/opt/conda/lib/python3.7/site-packages/IPython/core/magics/pylab.py:160: UserWarning: pylab import has clobbered these variables: ['f', 'clf']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [65]:
CONFIG = {
    'IMG_BASE_PATH': '/kaggle/input/hands-and-palm-images-dataset/Hands/Hands/'
}

m_class = 50

In [66]:
df_info = pd.read_csv('/kaggle/input/hands-and-palm-images-dataset/HandInfo.csv')

df_info = df_info[df_info.accessories == 0]
# df_info = df_info[df_info.aspectOfHand.isin(['palmar left'])]
df_info = df_info[df_info.id.isin(df_info.id.value_counts()[:m_class].index)]

le = preprocessing.LabelEncoder()
df_info['id'] = le.fit_transform(df_info['id'])

leH = preprocessing.LabelEncoder()
df_info['aspectOfHand'] = leH.fit_transform(df_info['aspectOfHand'])

In [67]:
df_info

,id,age,gender,skinColor,accessories,nailPolish,aspectOfHand,imageName,irregularities
1993,11,20,female,medium,0,0,1,Hand_0002300.jpg,0
1994,11,20,female,medium,0,0,1,Hand_0002301.jpg,0
1995,11,20,female,medium,0,0,1,Hand_0002302.jpg,0
1996,11,20,female,medium,0,0,1,Hand_0002303.jpg,0
1997,11,20,female,medium,0,0,1,Hand_0002304.jpg,0
...,...,...,...,...,...,...,...,...,...
11071,45,22,female,fair,0,0,2,Hand_0011740.jpg,0
11072,45,22,female,fair,0,0,2,Hand_0011741.jpg,0
11073,45,22,female,fair,0,0,2,Hand_0011742.jpg,0
11074,45,22,female,fair,0,0,2,Hand_0011743.jpg,0


In [68]:
df_info.aspectOfHand.value_counts()

3    1110
1    1040
2    1007
0     942
Name: aspectOfHand, dtype: int64

In [69]:
leH.inverse_transform(df_info['aspectOfHand'])

array(['dorsal right', 'dorsal right', 'dorsal right', ..., 'palmar left',
       'palmar left', 'palmar left'], dtype=object)

In [70]:
leH.classes_

array(['dorsal left', 'dorsal right', 'palmar left', 'palmar right'],
      dtype=object)

In [71]:
subjects_ids = df_info.id.unique()

df_info_list = []
for i in subjects_ids:
    df_i = df_info[df_info.id == i].head(10)
    df_info_list.extend(df_i.to_numpy())
    
df_info2 = pd.DataFrame(df_info_list)
df_info2.columns = df_info.columns
# df_info2 = df_info2[['id', 'imageName']]

In [72]:
def show_image(path: str):    
    img = mpimg.imread(path)
    imgplot = plt.imshow(img)
    plt.show()

In [73]:
X_train, X_test, y_train, y_test = train_test_split(df_info2, 
                                                    df_info2['id'], 
                                                    test_size=0.2, 
                                                    random_state=42, 
                                                    shuffle=True, 
                                                    stratify=df_info2['id'])
# X_train, X_valid, y_train, y_valid = train_test_split(X_train,y_train,test_size=0.3,random_state=42,shuffle=True, stratify=y_train)

In [74]:
images_train_list = []

for path in X_train.imageName:    
    image = tf.keras.preprocessing.image.load_img(CONFIG['IMG_BASE_PATH'] + os.sep + path)
    image_arr = tf.keras.preprocessing.image.img_to_array(image)
    image_arr = tf.image.resize(image_arr,(224, 224)).numpy()
    # image_arr = (255 - image_arr) / 255
    image_arr /= 255.
    
    # src = cv2.imread(CONFIG['IMG_BASE_PATH'] + os.sep + path)
    # dst_sketch, dst_color_sketch = cv2.pencilSketch(src, sigma_s=50, sigma_r=0.05, shade_factor=0.05)
    # dst_water_color = cv2.stylization(src, sigma_s=50, sigma_r=0.05)
    # image_arr = dst_water_color / 255.
    
    images_train_list.append(image_arr)

In [75]:
images_valid_list = []

for path in X_test.imageName:
    image = tf.keras.preprocessing.image.load_img(CONFIG['IMG_BASE_PATH'] + os.sep + path)
    image_arr = tf.keras.preprocessing.image.img_to_array(image)
    image_arr = tf.image.resize(image_arr,(224, 224)).numpy()
    # image_arr = (255 - image_arr) / 255
    image_arr /= 255
    
    images_valid_list.append(image_arr)

# MODEL

In [76]:
base_model = VGG16(weights='imagenet', include_top=True)

x = base_model.get_layer('fc2').output

model = models.Model(inputs=base_model.input, outputs=x)

In [77]:
train_input_vvg16 = preprocess_input(np.array(images_train_list))
valid_input_vvg16 = preprocess_input(np.array(images_valid_list))

train_features = model.predict(train_input_vvg16)
valid_features = model.predict(valid_input_vvg16)

train_features = pd.DataFrame(train_features)
valid_features = pd.DataFrame(valid_features)

In [78]:
train_features = pd.concat([train_features, X_train['aspectOfHand'].reset_index().drop(['index'], axis=1)], axis=1)
valid_features = pd.concat([valid_features, X_test['aspectOfHand'].reset_index().drop(['index'], axis=1)], axis=1)

In [79]:
from sklearn import svm
clf = svm.SVC()
clf.fit(train_features, y_train)

/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,


SVC()

In [89]:
preds = clf.predict(valid_features)

/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,


In [80]:
from sklearn.metrics import accuracy_score

accuracy_score(y_test, clf.predict(valid_features))

/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,


0.77

In [91]:
X_test

,id,age,gender,skinColor,accessories,nailPolish,aspectOfHand,imageName,irregularities
331,46,21,female,medium,0,0,1,Hand_0007918.jpg,0
54,32,20,female,medium,0,0,1,Hand_0002903.jpg,0
80,42,21,female,fair,0,0,0,Hand_0003592.jpg,0
236,18,21,female,medium,0,0,1,Hand_0006167.jpg,0
389,48,75,female,dark,0,0,0,Hand_0009418.jpg,0
...,...,...,...,...,...,...,...,...,...
430,39,24,male,medium,0,0,1,Hand_0010863.jpg,0
144,27,21,female,medium,0,0,1,Hand_0004675.jpg,0
8,11,20,female,medium,0,0,1,Hand_0002308.jpg,0
49,37,21,male,medium,0,0,1,Hand_0002853.jpg,0


In [90]:
for x in zip(y_test, preds):
    print(x)

(46, 46)
(32, 32)
(42, 38)
(18, 18)
(48, 48)
(5, 41)
(41, 41)
(36, 36)
(9, 9)
(15, 43)
(14, 14)
(19, 19)
(16, 16)
(31, 31)
(29, 29)
(19, 19)
(44, 44)
(6, 6)
(34, 34)
(36, 36)
(41, 41)
(26, 26)
(40, 40)
(4, 4)
(35, 35)
(33, 33)
(43, 43)
(28, 28)
(47, 47)
(26, 26)
(2, 39)
(18, 18)
(8, 8)
(38, 38)
(1, 1)
(3, 3)
(20, 20)
(21, 13)
(0, 0)
(11, 11)
(32, 32)
(48, 48)
(13, 43)
(25, 25)
(6, 6)
(29, 29)
(7, 7)
(17, 17)
(45, 45)
(35, 35)
(17, 17)
(31, 31)
(8, 34)
(23, 23)
(49, 49)
(14, 14)
(44, 44)
(37, 37)
(46, 46)
(30, 30)
(23, 23)
(28, 28)
(12, 43)
(4, 4)
(42, 38)
(22, 22)
(24, 24)
(16, 16)
(22, 22)
(5, 41)
(38, 38)
(0, 35)
(34, 34)
(39, 39)
(10, 9)
(40, 33)
(20, 20)
(2, 49)
(10, 9)
(9, 9)
(25, 25)
(30, 30)
(24, 24)
(15, 43)
(13, 13)
(43, 43)
(33, 33)
(12, 43)
(27, 27)
(45, 37)
(1, 24)
(3, 3)
(7, 7)
(49, 41)
(47, 47)
(39, 49)
(27, 27)
(11, 43)
(37, 37)
(21, 43)


In [81]:
X_test.to_csv('test.csv')

In [82]:
X_test.aspectOfHand.value_counts()

1    85
0    10
3     5
Name: aspectOfHand, dtype: int64

In [83]:
import pickle

In [84]:
with open('model_classification.pkl','wb') as f:
    pickle.dump(clf,f)